In [1]:
import glob
import h5py
import numpy as np
import os
import tqdm

from os import listdir
from os.path import isfile, join

In [3]:
with h5py.File('/ccn2/u/thekej/models/sgnn_physion/ocd/train_features.hdf5','r') as hf:
    print(hf.keys())
    filenames = [h.decode().split('/')[-1] for h in hf['filenames'][:]]
    #features_tester = hf['features_tester'][:]
    features = hf['features'][:]
    contacts = hf['contacts'][:]
    frame_labels = hf['frame_labels'][:]

<KeysViewHDF5 ['contacts', 'features', 'filenames', 'label', 'world_contacts']>


KeyError: "Unable to synchronously open object (object 'frame_labels' doesn't exist)"

In [ ]:
features.shape

In [7]:
with h5py.File('/ccn2/u/thekej/contact_data/train_contact_data.hdf5','r') as hf:
    label_contact_list = hf['label'][:]
    contacts_contact_list = hf['contacts'][:]  
    lengths_contact_list = hf['lengths'][:]
    filenames_contact_list = [d.decode() for d in hf['filenames'][:]]

In [8]:
label_new = []
filenames_new = []
features_new = []

In [9]:
for i, feats in enumerate(features_tester):
    # 1 frame, 3 frames, 7 frames,
    # 40, 120, 280
    # 1, 12, 28
    n = 28  # Number of contiguous frames to sample
    filename = filenames[i]

    if filename in filenames_contact_list:
        test_contact_index = filenames_contact_list.index(filename)
        length = lengths_contact_list[test_contact_index]
        lbl_ctct = label_contact_list[test_contact_index]
        total_frames = min(length - 45, 225 - 45)
        if total_frames - n < 0:
            continue
        # Determine the maximum starting point
        for _ in range(10):
            filenames_new += [filename]
            max_start = total_frames - n
            # Randomly select a starting frame
            start_frame = np.random.randint(0, max_start + 1)
            # Extract n contiguous frames
            sampled_frames = feats[start_frame:start_frame + n]
            features_new += [sampled_frames]
            label_new += [lbl_ctct[start_frame:start_frame + n]]

In [15]:
dt = h5py.special_dtype(vlen=str)

print('save 1')
with h5py.File('/ccn2/u/thekej/contact_data/sgnn_tester_feats_train_7_frame.hdf5','w') as hf:
    hf.create_dataset("label", data=label_new)
    hf.create_dataset("features", data=np.stack(features_new))
    hf.create_dataset("filenames", data=filenames_new, dtype=dt)

save 1


In [16]:
np.stack(features_new).shape

(55940, 28, 12, 512)

In [17]:
len(filenames_new)

55940

In [28]:
c = 0
for i in range(len(label_new)):
    if sum(label_new[i]) > 0:
        c+=1

In [29]:
c

8388